In [1]:
from kerchunk.hdf import SingleHdf5ToZarr
from kerchunk.combine import MultiZarrToZarr
import fsspec
import os
import ujson
from pathlib import Path
import xarray as xr
import numpy as np

In [9]:
fs = fsspec.filesystem('s3')
fs2 = fsspec.filesystem('')
dirs = [
    'northwest_atlantic/full_domain/seasonal_reforecast/monthly/raw/r20250212/',
    'northwest_atlantic/full_domain/seasonal_reforecast/monthly/regrid/r20250212/',
]

In [10]:
def fn_to_time(index, fs, var, fn):
    import re
    import datetime
    subst = re.search(r'.*i([0-9][0-9][0-9][0-9][0-9][0-9]).json', fn).groups()[0]
    subst = subst + '01000000.000000'
    return datetime.datetime.strptime(subst, '%Y%m%d%H%M%S.%f')

In [13]:
for directory in dirs:
    flist = fs.glob(f's3://noaa-oar-cefi-regional-mom6-pds/{directory}[s-z]*.json')
    flist = ['s3://' + s for s in flist]
    if 'raw' in dir:
        identical_dims = ['yh', 'xh', 'lead', 'member'] # raw
    else:
        identical_dims = ['lat', 'lon', 'lead', 'member'] #regrid
    os.makedirs(directory, exist_ok=True)
    mzz = MultiZarrToZarr(flist,  
        coo_map={'init_time': fn_to_time},
        coo_dtypes={'init_time': np.dtype('M8[ns]')},
        concat_dims=['init_time'],
        identical_dims = identical_dims
    )
    d = mzz.translate()
    with fs2.open(f'{directory}/all.json', 'wb') as f:
        f.write(ujson.dumps(d).encode())